In [1]:

import cv2


cap = cv2.VideoCapture(0)  # 0 means default webcam
while True:
    success, frame = cap.read()
    
    if not success:
        break

    cv2.imshow("Webcam", frame)
# Press 'spacebar' to quit
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
import mediapipe as mp
print(mp.__version__)


0.10.21


In [7]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Create Hands object
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    success, frame = cap.read()
    if not success:
        break
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(rgb_frame)

    # Draw hand landmarks
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS
            )
    cv2.imshow("Hand Tracking", frame)


    if cv2.waitKey(1) & 0xFF == ord(' '):
        break
cap.release()
cv2.destroyAllWindows()

In [9]:
import cv2
import mediapipe as mp
import math
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

#Pycaw Setup
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volRange = volume.GetVolumeRange() 
minVol = volRange[0]
maxVol = volRange[1]

#MediaPipe Setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lmList = []
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
            
            if lmList:
                x1, y1 = lmList[4][1], lmList[4][2]
                x2, y2 = lmList[8][1], lmList[8][2] 
                cv2.circle(img, (x1, y1), 10, (255, 0, 255), cv2.FILLED)
                cv2.circle(img, (x2, y2), 10, (255, 0, 255), cv2.FILLED)
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
                
                # Calculate distance using Eucleadean Distance
                length = math.hypot(x2 - x1, y2 - y1)
                
                # Map length (50-300 pixels) to Volume (-65 to 0)
                vol = np.interp(length, [50, 250], [minVol, maxVol])
                volume.SetMasterVolumeLevel(vol, None)
                
              

    cv2.imshow("Volume Control", img)
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

cap.release()
cv2.destroyAllWindows()

In [11]:
#Adding Volume Bar to previous Code
import cv2
import mediapipe as mp
import math
import numpy as np
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            lmList = []
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
            
            if lmList:
                x1, y1 = lmList[4][1], lmList[4][2]
                x2, y2 = lmList[8][1], lmList[8][2]
                cv2.circle(img, (x1, y1), 10, (255, 0, 255), cv2.FILLED)
                cv2.circle(img, (x2, y2), 10, (255, 0, 255), cv2.FILLED)
                cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
                length = math.hypot(x2 - x1, y2 - y1)
                vol = np.interp(length, [50, 250], [minVol, maxVol])
                volume.SetMasterVolumeLevel(vol, None)
                
                # Visual Volume Bar
                volBar = np.interp(length, [50, 250], [400, 150])
                cv2.rectangle(img, (50, 150), (85, 400), (0, 255, 0), 3)
                cv2.rectangle(img, (50, int(volBar)), (85, 400), (0, 255, 0), cv2.FILLED)

    cv2.imshow("Volume Control", img)
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pyautogui
import math

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, model_complexity=1)
draw_utils = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    success, frame = cap.read()
    if not success: break
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = hands.process(rgb_frame)

    if output.multi_hand_landmarks:
        for hand in output.multi_hand_landmarks:
            lm = hand.landmark

            xb, yb = lm[9].x, lm[9].y
            hand_scale = math.hypot(xb - xw, yb - yw)
            x1, y1 = int(lm[4].x * w), int(lm[4].y * h)
            x2, y2 = int(lm[8].x * w), int(lm[8].y * h)
            finger_gap = math.hypot(lm[8].x - lm[4].x, lm[8].y - lm[4].y)

            gap_ratio = finger_gap / (hand_scale + 0.001)
            fingers_open = []
            for tip, pip in zip([8, 12, 16, 20], [6, 10, 14, 18]):
                fingers_open.append(lm[tip].y < lm[pip].y) 
            if all(f == False for f in fingers_open):
                pyautogui.press("volumemute")
                cv2.putText(frame, "MUTED", (w//2-50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                cv2.waitKey(500)
            elif all(f == True for f in fingers_open):
                cv2.putText(frame, "VOLUME LOCKED! EXITING...", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
                cv2.imshow("Hand Volume Control", frame)
                cv2.waitKey(2000) 
                cap.release()
                cv2.destroyAllWindows()
                exit()
            else:
                vol_level = np.interp(gap_ratio, [0.3, 1.2], [0, 100])
                pyautogui.run(f"setvolume {int(vol_level)}") 
                if gap_ratio > 0.9: pyautogui.press("volumeup")
                elif gap_ratio < 0.4: pyautogui.press("volumedown")
                    
                cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)
                cv2.putText(frame, f"Vol Level: {int(vol_level)}%", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)

            draw_utils.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Hand Volume Control", frame)
    if cv2.waitKey(1) & 0xFF == ord(' '): break

cap.release()
cv2.destroyAllWindows()

NameError: name 'xw' is not defined

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pyautogui
import math

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, model_complexity=0)
draw_utils = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    success, frame = cap.read()
    if not success: break
    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = hands.process(rgb_frame)

    if output.multi_hand_landmarks:
        for hand in output.multi_hand_landmarks:
            lm = hand.landmark

            xw, yw = lm[0].x, lm[0].y  
            hand_scale = math.hypot(xb - xw, yb - yw)

            x1, y1 = int(lm[4].x * w), int(lm[4].y * h)
            x2, y2 = int(lm[8].x * w), int(lm[8].y * h)

            finger_gap = math.hypot(lm[8].x - lm[4].x, lm[8].y - lm[4].y)
            gap_ratio = finger_gap / (hand_scale + 0.001)

            fingers_open = []
            for tip, pip in zip([8, 12, 16, 20], [6, 10, 14, 18]):
                fingers_open.append(lm[tip].y < lm[pip].y) )
            if not any(fingers_open):
                pyautogui.press("volumemute")
                cv2.putText(frame, "MUTED", (w//2-50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
                cv2.waitKey(500) 
            elif all(fingers_open): 
                cv2.putText(frame, "EXITING...", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
                cv2.imshow("Hand Volume Control", frame)
                cv2.waitKey(1000) 
                cap.release()
                cv2.destroyAllWindows()
                exit())
            else:
                vol_level = np.interp(gap_ratio, [0.2, 1.0], [0, 100])
                
                if gap_ratio > 0.8: 
                    pyautogui.press("volumeup")
                elif gap_ratio < 0.3: 
                    pyautogui.press("volumedown")
                    
                cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 0), 3)
                cv2.putText(frame, f"Vol: {int(vol_level)}%", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)

            draw_utils.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Hand Volume Control", frame)
    if cv2.waitKey(1) & 0xFF == ord(' '): break 

cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Initialize Pycaw
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.8, model_complexity=1)
draw_utils = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    success, frame = cap.read()
    if not success: break
    
    frame = cv2.flip(frame, 1) 
    h, w, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = hands.process(rgb_frame)

    if output.multi_hand_landmarks:
        for hand in output.multi_hand_landmarks:
            lm = hand.landmark
            palmW = math.hypot(lm[5].x - lm[0].x, lm[5].y - lm[0].y)
            finger_gap = math.hypot(lm[8].x - lm[4].x, lm[8].y - lm[4].y)
            ratio = finger_gap / (palmW + 0.001)

        
            tips = [lm[8].y, lm[12].y, lm[16].y, lm[20].y]
            joints = [lm[6].y, lm[10].y, lm[14].y, lm[18].y]
            fingers_open = [tips[i] < joints[i] for i in range(4)]

            if not any(fingers_open):
                volume.SetMute(1, None)
                status = "MUTED"
                color = (0, 0, 255)

            elif all(fingers_open):
                status = "LOCKED & EXITING"
                cv2.putText(frame, status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow("Volume Control", frame)
                cv2.waitKey(1000)
                cap.release()
                cv2.destroyAllWindows()
                exit()
            
            else:
                volume.SetMute(0, None)
                vol_level = np.interp(ratio, [0.2, 1.1], [0.0, 1.0])
                volume.SetMasterVolumeLevelScalar(vol_level, None)
                
                status = f"Volume: {int(vol_level * 100)}%"
                color = (255, 255, 0)
                
            cv2.putText(frame, status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            draw_utils.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Volume Control", frame)
    if cv2.waitKey(1) & 0xFF == ord(' '): break

cap.release()
cv2.destroyAllWindows()